In [12]:
# 한컴 DocsConverter API - OAuth 인증 방식

import requests
import json
import base64
import os
from datetime import datetime

class HancomDocsConverterOAuth:
    def __init__(self, client_id, client_secret):
        """
        한컴 DocsConverter API - OAuth 인증
        
        Args:
            client_id: 한컴디벨로퍼에서 발급받은 Client ID
            client_secret: 한컴디벨로퍼에서 발급받은 Client Secret
        """
        self.client_id = client_id
        self.client_secret = client_secret
        self.base_url = "https://api.hancom.com"  # 실제 URL은 문서 확인 필요
        self.access_token = None
        
    def get_access_token(self):
        """
        OAuth2.0 방식으로 액세스 토큰 획득
        
        Returns:
            str: 액세스 토큰
        """
        try:
            # OAuth 토큰 요청
            token_url = f"{self.base_url}/oauth/token"
            
            # 인증 정보를 Base64로 인코딩
            credentials = f"{self.client_id}:{self.client_secret}"
            encoded_credentials = base64.b64encode(credentials.encode()).decode()
            
            headers = {
                'Authorization': f'Basic {encoded_credentials}',
                'Content-Type': 'application/x-www-form-urlencoded'
            }
            
            data = {
                'grant_type': 'client_credentials',
                'scope': 'docsconverter'  # 필요한 스코프
            }
            
            response = requests.post(token_url, headers=headers, data=data)
            
            if response.status_code == 200:
                token_data = response.json()
                self.access_token = token_data.get('access_token')
                print("액세스 토큰 획득 성공!")
                return self.access_token
            else:
                print(f"토큰 획득 실패: {response.status_code} - {response.text}")
                return None
                
        except Exception as e:
            print(f"토큰 획득 중 오류: {e}")
            return None
    
    def convert_file_to_hwp(self, input_file_path, output_hwp_path, input_format='docx'):
        """
        파일을 HWP로 변환
        
        Args:
            input_file_path: 입력 파일 경로 (DOCX, RTF 등)
            output_hwp_path: 출력 HWP 파일 경로
            input_format: 입력 파일 형식 ('docx', 'rtf' 등)
            
        Returns:
            bool: 변환 성공 여부
        """
        # 먼저 액세스 토큰 획득
        if not self.access_token:
            if not self.get_access_token():
                return False
        
        try:
            # 파일을 Base64로 인코딩
            with open(input_file_path, 'rb') as f:
                file_content = base64.b64encode(f.read()).decode('utf-8')
            
            # API 요청 데이터
            convert_url = f"{self.base_url}/docsconverter/v1/convert"
            
            headers = {
                'Authorization': f'Bearer {self.access_token}',
                'Content-Type': 'application/json'
            }
            
            payload = {
                'inputFormat': input_format,
                'outputFormat': 'hwp',
                'fileData': file_content,
                'options': {
                    'preserveFormatting': True,
                    'embedFonts': True
                }
            }
            
            # API 호출
            response = requests.post(convert_url, headers=headers, json=payload, timeout=60)
            
            if response.status_code == 200:
                result = response.json()
                
                if result.get('success', False):
                    # 변환된 HWP 파일 저장
                    hwp_data = base64.b64decode(result.get('fileData', ''))
                    
                    with open(output_hwp_path, 'wb') as f:
                        f.write(hwp_data)
                    
                    print(f"HWP 변환 성공: {output_hwp_path}")
                    return True
                else:
                    error_msg = result.get('message', '알 수 없는 오류')
                    print(f"변환 실패: {error_msg}")
                    return False
            else:
                print(f"API 호출 실패: {response.status_code} - {response.text}")
                
                # 토큰 만료일 수 있으므로 재시도
                if response.status_code == 401:
                    print("토큰 만료 가능, 재인증 시도...")
                    self.access_token = None
                    return self.convert_file_to_hwp(input_file_path, output_hwp_path, input_format)
                
                return False
                
        except Exception as e:
            print(f"파일 변환 중 오류: {e}")
            return False

# 기존 코드와 통합된 HWP 생성 함수
def create_hwp_with_hancom_api(content, output_path, client_id, client_secret):
    """
    한컴 API를 사용하여 실제 HWP 파일 생성
    
    Args:
        content: 문서 내용 {'title': str, 'body': str}
        output_path: HWP 출력 경로
        client_id: 한컴 Client ID
        client_secret: 한컴 Client Secret
        
    Returns:
        bool: 성공 여부
    """
    
    # 1단계: DOCX 파일 생성
    docx_path = output_path.replace('.hwp', '_temp_for_api.docx')
    
    try:
        # 기존 Word 생성 코드 활용
        from docx import Document
        from docx.shared import Pt
        from docx.enum.text import WD_ALIGN_PARAGRAPH
        
        doc_word = Document()
        
        # 제목 추가
        title_para = doc_word.add_heading(content['title'], 0)
        title_para.alignment = WD_ALIGN_PARAGRAPH.CENTER
        doc_word.add_paragraph()
        
        # 본문 처리
        body_lines = content['body'].split('\n')
        for line in body_lines:
            if line.strip() == "---------------------------------------------------------------------":
                p = doc_word.add_paragraph(line)
                p.alignment = WD_ALIGN_PARAGRAPH.CENTER
            elif "##HEADING_START##" in line and "##HEADING_END##" in line:
                heading = line.replace('##HEADING_START##', '').replace('##HEADING_END##', '')
                p = doc_word.add_paragraph()
                run = p.add_run(heading)
                run.bold = True
                run.underline = True
                run.font.size = Pt(14)
            elif line.strip():
                p = doc_word.add_paragraph(line)
                p.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY
            else:
                doc_word.add_paragraph()
        
        # 하단 링크 추가
        doc_word.add_paragraph()
        link_paragraph = doc_word.add_paragraph()
        run1 = link_paragraph.add_run("더 많은 기도문 보러가기:")
        run1.bold = True
        run1.font.size = Pt(14)
        link_paragraph.add_run("\n")
        run2 = link_paragraph.add_run("[대표 기도문 나눔터] https://prayer-church.co.kr/")
        run2.bold = True
        run2.font.size = Pt(14)
        link_paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
        
        # DOCX 저장
        doc_word.save(docx_path)
        print(f"임시 DOCX 파일 생성: {docx_path}")
        
        # 2단계: 한컴 API를 사용하여 DOCX → HWP 변환
        converter = HancomDocsConverterOAuth(client_id, client_secret)
        success = converter.convert_file_to_hwp(docx_path, output_path, 'docx')
        
        # 임시 파일 정리
        if os.path.exists(docx_path):
            os.remove(docx_path)
            print(f"임시 파일 삭제: {docx_path}")
        
        return success
        
    except ImportError as e:
        print(f"필요한 라이브러리가 설치되지 않았습니다: {e}")
        print("설치 명령어: pip install requests python-docx")
        return False
    except Exception as e:
        print(f"한컴 API를 사용한 HWP 생성 실패: {e}")
        
        # 임시 파일 정리
        if os.path.exists(docx_path):
            os.remove(docx_path)
        
        return False

# 기존 create_hwp_file 함수 수정
def create_hwp_file_with_api(content, output_path):
    """
    한컴 API를 사용하여 HWP 파일 생성 (Client ID/Secret 활용)
    """
    
    # 한컴 API 인증 정보 (실제 값으로 교체)
    CLIENT_ID = "SzOfBHd8PTcGvYR9S9IT"
    CLIENT_SECRET = "SEAIwuQmFo"  # 전체 시크릿 키
    
    print("한컴 API를 사용하여 HWP 파일 생성 시도...")
    
    # 한컴 API로 HWP 생성 시도
    if create_hwp_with_hancom_api(content, output_path, CLIENT_ID, CLIENT_SECRET):
        print(f"✅ HWP 파일 생성 성공: {output_path}")
        return
    
    # API 실패시 대안 방법
    print("❌ 한컴 API 실패, 대안 방법으로 파일 생성...")
    
    # RTF 파일 생성
    rtf_path = output_path.replace('.hwp', '.rtf')
    create_enhanced_rtf_file(content, rtf_path)
    
    # DOCX 파일 생성
    docx_path = output_path.replace('.hwp', '.docx')
    create_word_file_simple(content, docx_path)
    
    print(f"\n생성된 파일들:")
    print(f"- RTF: {rtf_path}")
    print(f"- DOCX: {docx_path}")
    print("\n💡 온라인 변환 서비스에서 HWP로 변환하세요:")
    print("- https://anyconv.com/ko/docx-to-hwp-byeonhwangi/")

def create_word_file_simple(content, output_path):
    """간단한 DOCX 파일 생성"""
    try:
        from docx import Document
        from docx.shared import Pt
        from docx.enum.text import WD_ALIGN_PARAGRAPH
        
        doc = Document()
        
        # 제목
        title = doc.add_heading(content['title'], 0)
        title.alignment = WD_ALIGN_PARAGRAPH.CENTER
        
        # 본문
        doc.add_paragraph()
        body_lines = content['body'].split('\n')
        for line in body_lines:
            if line.strip():
                doc.add_paragraph(line)
        
        # 링크
        doc.add_paragraph()
        link_para = doc.add_paragraph("더 많은 기도문 보러가기:")
        link_para.alignment = WD_ALIGN_PARAGRAPH.CENTER
        link_para = doc.add_paragraph("[대표 기도문 나눔터] https://prayer-church.co.kr/")
        link_para.alignment = WD_ALIGN_PARAGRAPH.CENTER
        
        doc.save(output_path)
        print(f"DOCX 파일 생성 완료: {output_path}")
        
    except Exception as e:
        print(f"DOCX 생성 실패: {e}")

# 사용 예시
if __name__ == "__main__":
    # 테스트 콘텐츠
    test_content = {
        'title': '5월 주일 대표 기도문',
        'body': '하나님 아버지, 감사드립니다.\n\n##HEADING_START##중보기도##HEADING_END##\n\n가족과 이웃을 위해 기도드립니다.\n\n아멘.'
    }
    
    # HWP 파일 생성
    create_hwp_file_with_api(test_content, 'test_output.hwp')

한컴 API를 사용하여 HWP 파일 생성 시도...
임시 DOCX 파일 생성: test_output_temp_for_api.docx
토큰 획득 실패: 404 - {"timestamp":"2025-05-14T04:58:43.420+00:00","path":"/oauth/token","status":404,"error":"Not Found","message":null,"requestId":"9c1e1e5d-370268"}
임시 파일 삭제: test_output_temp_for_api.docx
❌ 한컴 API 실패, 대안 방법으로 파일 생성...
RTF 파일 생성 완료: test_output.rtf
DOCX 파일 생성 완료: test_output.docx

생성된 파일들:
- RTF: test_output.rtf
- DOCX: test_output.docx

💡 온라인 변환 서비스에서 HWP로 변환하세요:
- https://anyconv.com/ko/docx-to-hwp-byeonhwangi/


In [9]:
pip install unoconv


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
